In [2]:
import numpy as np
import itertools
from typing import Literal, Tuple
from collections import Counter

In [3]:
from phevaluator.evaluator import evaluate_cards

In [4]:
from deck.card import Rank
from deck.deck import Deck

In [5]:
def sort_hand(hand: np.ndarray) -> np.ndarray:
    return np.array(sorted(hand, reverse=True, key=lambda x: (x.get_rank(), x.get_tag())))


In [6]:
def is_a_pair(hand: np.ndarray) -> bool:
    return hand[0].get_rank() == hand[1].get_rank()

def compare_starting_hands(hand_one: np.ndarray, hand_two: np.ndarray) -> int:
    """
    Return 1 if hand one is stronger than hand two, -1 if hand two stronger, 0 if equal
    """
    is_hand_one_a_pair = is_a_pair(hand_one)
    is_hand_two_a_pair = is_a_pair(hand_two)
    if is_hand_one_a_pair and not is_hand_two_a_pair:
        return 1
    elif not is_hand_one_a_pair and is_hand_two_a_pair:
        return -1
    elif is_hand_one_a_pair and is_hand_two_a_pair:
        rank_one = hand_one[0].get_rank()
        rank_two = hand_two[0].get_rank()
        if rank_one == rank_two:
            return 0
        if rank_one > rank_two:
            return 1
        else:
            return -1
    else:
        hand_one = sort_hand(hand_one)
        hand_two = sort_hand(hand_two)
        if hand_one[0].get_rank() > hand_two[0].get_rank():
            return 1
        elif hand_one[0].get_rank() < hand_two[0].get_rank():
            return -1
        else:
            if hand_one[1].get_rank() > hand_two[1].get_rank():
                return 1
            elif hand_one[1].get_rank() < hand_two[1].get_rank():
                return -1
            else:
                return 0

In [22]:
def compute_starting_effective_hand_strengths(iterations: int, suited=False):
    if suited:
        name = "s"
        second_suit = "c"
        starting_hands = itertools.combinations(list(Rank)[::-1], 2)
    else:
        name = "o"
        second_suit = "s"
        starting_hands = itertools.combinations_with_replacement(list(Rank)[::-1], 2) 
    effective_hand_strengths = {}
    for starting_hand in starting_hands:
        deck = Deck()
        cards = np.concatenate((deck.draw_card_by_tag(str(starting_hand[0].value) + "c"), deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit)))
        wins = 0
        positive_potential = 0
        negative_potential = 0
        for _ in range(iterations):
            deck.shuffle()
            opponent_cards = deck.draw_cards(2)
            initial_advantage = compare_starting_hands(sort_hand(cards), sort_hand(opponent_cards))
            flop = deck.draw_cards(3)
            turn = deck.draw_cards(1)
            river = deck.draw_cards(1)
            my_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((cards, flop, turn, river))])
            opponent_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((opponent_cards, flop, turn, river))])
            if my_strength < opponent_strength:
                wins += 1
                if initial_advantage == -1:
                    positive_potential += 1 #won the hand with initial disadvantage
                elif initial_advantage == 0:
                    positive_potential += 0.5
            elif my_strength == opponent_strength: # draw
                wins += 0.5
                if initial_advantage == -1:
                    positive_potential += 0.5
                elif initial_advantage == 1:
                    negative_potential += 0.5
            else: 
                if initial_advantage == 1: #lost the hand with initial advantage
                    negative_potential += 1
                elif initial_advantage == 0:
                    negative_potential += 0.5
            deck.return_cards(opponent_cards)
            deck.return_cards(flop)
            deck.return_cards(turn)
            deck.return_cards(river)
        effective_hand_strengths[str(starting_hand[0].value) + str(starting_hand[1].value) + name] = (wins * (iterations - negative_potential) + (iterations - wins) * positive_potential) / iterations ** 2
    return effective_hand_strengths

In [ ]:
iterations = 10000
suited_effective_hand_strength = compute_starting_effective_hand_strengths(iterations, True)
offsuit_effective_hand_strength = compute_starting_effective_hand_strengths(iterations, False)
effective_hand_strength = {
    **suited_effective_hand_strength,
    **offsuit_effective_hand_strength
}
#for k,v in sorted(effective_hand_strength.items(), key=lambda item: item[1], reverse=True):
#    print(k, v)

In [ ]:
import json
with open("poker/ehs/preflop/ehs.json", 'w') as f:
    json.dump(effective_hand_strength, f)

In [7]:
def compare_hands_after_flop(hand_one: np.ndarray, hand_two: np.ndarray) -> int:
    hand_one_strength = evaluate_cards(*[card.get_tag() for card in hand_one])
    hand_two_strength = evaluate_cards(*[card.get_tag() for card in hand_two])
    if hand_one_strength < hand_two_strength:
        return 1
    elif hand_one_strength == hand_one_strength:
        return 0
    else:
        return -1

In [8]:
def calculate_post_flop_result(
    cards,
    opponent_cards,
    flop,
    turn,
    river,
    wins,
    positive_potential,
    negative_potential,
    round: Literal["flop", "turn"] = "flop",
):
    if round == "flop":
        initial_advantage = compare_hands_after_flop(
            np.concatenate((cards, flop)),
            np.concatenate((opponent_cards, flop)),
        )
    else:
        initial_advantage = compare_hands_after_flop(
            np.concatenate((cards, flop, turn)),
            np.concatenate((opponent_cards, flop, turn)),
        )
    final_result = compare_hands_after_flop(
        np.concatenate((cards, flop, turn, river)),
        np.concatenate((opponent_cards, flop, turn, river)),
    )
    if final_result == 1:
        wins += 1
        if initial_advantage == -1:
            positive_potential += 1  # won the hand with initial disadvantage
        elif initial_advantage == 0:
            positive_potential += 0.5
    elif final_result == 0:  # draw
        wins += 0.5
        if initial_advantage == -1:
            positive_potential += 0.5
        elif initial_advantage == 1:
            negative_potential += 0.5
    else:
        if initial_advantage == 1:  # lost the hand with initial advantage
            negative_potential += 1
        elif initial_advantage == 0:
            negative_potential += 0.5
    return wins, positive_potential, negative_potential

In [ ]:
def perform_n_postflop_iterations(iterations: int, deck: Deck, cards: np.ndarray, flop:np.ndarray) -> Tuple[float, float, float]:
    wins = 0
    positive_potential = 0
    negative_potential = 0
    for _ in range(iterations):
        deck.shuffle()
        opponent_cards = deck.draw_cards(2)
        turn = deck.draw_cards(1)
        river = deck.draw_cards(1)
        wins, positive_potential, negative_potential = (
            calculate_post_flop_result(
                cards,
                opponent_cards,
                flop,
                turn,
                river,
                wins,
                positive_potential,
                negative_potential,
            )
        )
        deck.return_cards(opponent_cards)
        deck.return_cards(turn)
        deck.return_cards(river)
    return wins, positive_potential, negative_potential

In [ ]:
def perform_n_postturn_iterations(iterations: int, deck: Deck, cards: np.ndarray, flop:np.ndarray, turn: np.ndarray) -> Tuple[float, float, float]:
    wins = 0
    positive_potential = 0
    negative_potential = 0
    for _ in range(iterations):
        deck.shuffle()
        opponent_cards = deck.draw_cards(2)
        river = deck.draw_cards(1)
        wins, positive_potential, negative_potential = (
            calculate_post_flop_result(
                cards,
                opponent_cards,
                flop,
                turn,
                river,
                wins,
                positive_potential,
                negative_potential,
                "turn"
            )
        )
        deck.return_cards(opponent_cards)
        deck.return_cards(river)
    return wins, positive_potential, negative_potential

In [9]:
def perform_n_postriver_iterations(iterations: int, deck: Deck, cards: np.ndarray, flop:np.ndarray, turn: np.ndarray, river:np.ndarray) -> float:
    my_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((cards, flop, turn, river))])
    wins = 0
    for _ in range(iterations):
        deck.shuffle()
        opponent_cards = deck.draw_cards(2)
        opponent_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((opponent_cards, flop, turn, river))])
        if my_strength < opponent_strength:
            wins += 1
        elif my_strength == opponent_strength:
            wins += 0.5
        deck.return_cards(opponent_cards)
    return wins / iterations
        

In [ ]:
def compute_mono_flop_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    type = "_mono_"
    suited = "s"
    first_suit = "c"
    if made_flush:
        flush = "f"
        second_suit = first_suit
    else:
        flush = "n"
        second_suit = "s"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        possible_combinations = itertools.combinations(rank_list[::-1], 3)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[1].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                + type
                + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_mono_and_flush = compute_mono_flop_effective_hand_strengths_for_suited_cards(500)
flop_effective_hand_strength_mono_and_no_flush = compute_mono_flop_effective_hand_strengths_for_suited_cards(500, made_flush=False)

In [ ]:
#for k,v in sorted(flop_effective_hand_strength_mono_and_flush.items(), key=lambda item: item[1], reverse=True):
#    print(k, round(v, 3))

In [ ]:
#for k,v in sorted(flop_effective_hand_strength_mono_and_no_flush.items(), key=lambda item: item[1], reverse=True):
#    print(k, round(v, 3))

In [10]:
def create_a_list_of_lower_ranks(rank: Rank) -> list:
    return [x for x in list(Rank) if x < rank]

In [ ]:
def compute_21_flop_effective_hand_strengths_for_suited_cards(
    iterations: int, flush_draw=True
):
    type = "_twoone_"
    first_suit = "c"
    suited = "s"
    if flush_draw:
        flush = "d"
        second_suit = first_suit
        third_suit = "s"
    else:
        flush = "n"
        second_suit = "s"
        third_suit = "d"

    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if flush_draw:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        possible_combinations = itertools.combinations(rank_list[::-1], 2)
        for combination in possible_combinations:
            first_card = deck.draw_card_by_tag(str(combination[0].value) + second_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + second_suit)
            third_suit_rank_list = [
                combination[0],
                combination[1],
            ] + create_a_list_of_lower_ranks(combination[1])[::-1]
            if flush_draw:
                if starting_hand[1] > combination[1]:
                    third_suit_rank_list = [starting_hand[0], starting_hand[1]] + third_suit_rank_list
                elif starting_hand[0] > combination[1]:
                    third_suit_rank_list = [starting_hand[0]] + third_suit_rank_list
            for rank in third_suit_rank_list:
                third_card = deck.draw_card_by_tag(str(rank.value) + third_suit)
                flop = np.concatenate(
                    (
                        first_card,
                        second_card,
                        third_card
                    )
                )
                flop = sort_hand(flop)
                wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
                deck.return_cards(third_card)
                effective_hand_strengths[
                    str(starting_hand[0].value)
                    + str(starting_hand[1].value)
                    + suited
                    + "_"
                    + str(flop[0].rank.value)
                    + str(flop[1].rank.value)
                    + str(flop[2].rank.value)
                    + type
                    + flush
                ] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations**2
            deck.return_cards(first_card)
            deck.return_cards(second_card)
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_two_one_and_flush_draw = compute_21_flop_effective_hand_strengths_for_suited_cards(500)
flop_effective_hand_strength_two_one_no_flush_draw = compute_21_flop_effective_hand_strengths_for_suited_cards(500, False)

In [ ]:
#for k,v in flop_effective_hand_strength_two_one_and_flush_draw.items():
#    print(k, v)
#for k,v in sorted(flop_effective_hand_strength_two_one_and_flush_draw.items(), key=lambda item: item[1], reverse=True):
#    print(k, round(v, 2))

In [ ]:
def compute_rainbow_flop_effective_hand_strengths_for_suited_cards(iterations: int):
    name = "s"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations_with_replacement(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + fourth_suit),
                )
            )
            wins, positive_potential, negative_potential =  perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations_sq
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_ranibow_suited = compute_rainbow_flop_effective_hand_strengths_for_suited_cards(500)

In [ ]:
#for k,v in flop_effective_hand_strength_ranibow_suited.items():
#    print(k, v)

In [ ]:
def compute_mono_flop_effective_hand_strengths_for_offsuit_cards_with_flush_draw(iterations: int, with_higher_card = True):
    name = "o"
    type = "_mono_"
    flush = "d_higher" if with_higher_card else "d_lower" 
    first_suit = "c"
    second_suit = "s"
    flush_suit = first_suit if with_higher_card else second_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        if with_higher_card:
            rank_list.remove(starting_hand[0])
        else:
            rank_list.remove(starting_hand[1])
        possible_combinations = itertools.combinations(rank_list[::-1], 3)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_offsuit_mono_flush_draw_with_higher = compute_mono_flop_effective_hand_strengths_for_offsuit_cards_with_flush_draw(500, True)
flop_effective_hand_strength_offsuit_mono_flush_draw_with_lower = compute_mono_flop_effective_hand_strengths_for_offsuit_cards_with_flush_draw(500, False)


In [ ]:
#for k,v in flop_effective_hand_strength_offsuit_mono_flush_draw_with_higher.items():
#    print(k, v)

In [ ]:
def compute_mono_flop_effective_hand_strengths_for_pairs_with_flush_draw(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "d"
    first_suit = "c"
    second_suit = "s"
    flush_suit = first_suit
    effective_hand_strengths = {}
    for rank_one in list(Rank)[::-1]:
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(rank_one.value) + first_suit),
                deck.draw_card_by_tag(str(rank_one.value) + second_suit),
            )
        )
        rank_list = list(Rank)
        rank_list.remove(rank_one)
        possible_combinations = itertools.combinations(rank_list[::-1], 3)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            if str(rank_one.value) + str(rank_one.value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + "t" + flush in effective_hand_strengths:
                raise ValueError(str(rank_one.value) + str(rank_one.value) + name + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + "t" + flush)
            effective_hand_strengths[
                str(rank_one.value) + str(rank_one.value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_pairs_mono_flush_draw = compute_mono_flop_effective_hand_strengths_for_pairs_with_flush_draw(500)
#for k,v in flop_effective_hand_strength_pairs_mono_flush_draw.items():
#    print(k, v)


In [ ]:
def compute_mono_flop_effective_hand_strengths_for_offsuit_cards_no_flush_draw(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        possible_combinations = itertools.combinations(rank_list[::-1], 3)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + third_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
def compute_mono_flop_effective_hand_strengths_for_pairs_no_flush_draw(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        possible_combinations = itertools.combinations(rank_list[::-1], 3)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + third_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strength_offsuit_mono_no_flush_draw = compute_mono_flop_effective_hand_strengths_for_offsuit_cards_no_flush_draw(500)
flop_effective_hand_strength_pairs_mono_no_flush_draw = compute_mono_flop_effective_hand_strengths_for_pairs_no_flush_draw(500)


In [ ]:
#for k,v in flop_effective_hand_strength_offsuit_mono_no_flush_draw.items():
#    print(k, v)

In [ ]:
def compute_21_flop_effective_hand_strengths_for_offsuit_cards_no_flush_draw(iterations: int):
    name = "o"
    flush = "n"
    type = "_twoone_"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    
        
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        possible_combinations = itertools.combinations(rank_list[::-1], 2)
        for combination in possible_combinations:
            third_suit_rank_list = [combination[0], combination[1]] + create_a_list_of_lower_ranks(combination[1])[::-1]
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for rank in third_suit_rank_list:
                third_card = deck.draw_card_by_tag(str(rank.value) + fourth_suit)
                flop = np.concatenate(
                    (
                        first_card,
                        second_card,
                        third_card,
                    )
                )
                flop = sort_hand(flop)
                wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
                deck.return_cards(third_card)
                effective_hand_strengths[
                    str(starting_hand[0].value) + str(starting_hand[1].value) + name +"_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
                ] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations**2
            deck.return_cards(first_card)
            deck.return_cards(second_card)
    return effective_hand_strengths

In [ ]:
def compute_21_flop_effective_hand_strengths_for_pairs_no_flush_draw(iterations: int):
    name = "o"
    flush = "n"
    type = "_twoone_"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    
        
    effective_hand_strengths = {}
    for rank_one in list(Rank)[::-1]:
        rank_two = rank_one
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(rank_one.value) + first_suit),
                deck.draw_card_by_tag(str(rank_two.value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            third_suit_rank_list = [combination[0], combination[1]] + create_a_list_of_lower_ranks(combination[1])[::-1]
            for rank in third_suit_rank_list:
                flop = np.concatenate(
                    (
                        deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                        deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                        deck.draw_card_by_tag(str(rank.value) + fourth_suit),
                    )
                )
                flop = sort_hand(flop)
                wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
                deck.return_cards(flop)
                effective_hand_strengths[
                    str(rank_one.value) + str(rank_two.value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
                ] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations**2
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strengths_21_for_offsuit_cards = compute_21_flop_effective_hand_strengths_for_offsuit_cards_no_flush_draw(500)
flop_effective_hand_strengths_21_for_pairs = compute_21_flop_effective_hand_strengths_for_pairs_no_flush_draw(500)
#for k,v in effective_hand_strengths_21_flop_for_offsuit_cards.items():
#    print(k, v)


In [ ]:
#for k,v in flop_effective_hand_strengths_21_for_pairs.items():
#    print(k, v)

In [ ]:
def compute_rainbow_flop_effective_hand_strengths_for_offsuit_cards(iterations: int):
    name = "o"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations_with_replacement(rank_list[::-1], 3):
            temp_suit = first_suit if combination[0] != starting_hand[0] else second_suit
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + temp_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + fourth_suit),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(starting_hand[0].value) + str(starting_hand[1].value) + name +"_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations_sq
    return effective_hand_strengths

In [ ]:
def compute_rainbow_flop_effective_hand_strengths_for_pairs(iterations: int):
    name = "o"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for rank_one in list(Rank)[::-1]:
        rank_two = rank_one
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(rank_one.value) + first_suit),
                deck.draw_card_by_tag(str(rank_two.value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations_with_replacement(rank_list[::-1], 3):
            if combination[0] == combination[2] and rank_one == combination[0]:
                continue
            if combination[0] == rank_one:
                temp_suit_one = third_suit
                temp_suit_two = fourth_suit
                temp_suit_three = first_suit
            else:
                temp_suit_one = first_suit
                temp_suit_two = third_suit
                temp_suit_three = fourth_suit
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + temp_suit_one),
                    deck.draw_card_by_tag(str(combination[1].value) + temp_suit_two),
                    deck.draw_card_by_tag(str(combination[2].value) + temp_suit_three),
                )
            )
            wins, positive_potential, negative_potential = perform_n_postflop_iterations(iterations, deck, cards, flop)
            deck.return_cards(flop)
            effective_hand_strengths[
                str(rank_one.value) + str(rank_two.value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + type + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations_sq
    return effective_hand_strengths

In [ ]:
flop_effective_hand_strengths_rainbow_for_offsuit_cards = compute_rainbow_flop_effective_hand_strengths_for_offsuit_cards(500)
flop_effective_hand_strengths_rainbow_for_pairs = compute_rainbow_flop_effective_hand_strengths_for_pairs(500)
#for k,v in flop_effective_hand_strengths_rainbow_for_offsuit_cards.items():
#    print(k, v)

In [ ]:
#for k,v in flop_effective_hand_strengths_rainbow_for_pairs.items():
#    print(k, v)

In [ ]:
effective_hand_strengths_postflop = {
    **flop_effective_hand_strength_mono_and_flush,
    **flop_effective_hand_strength_mono_and_no_flush,
    **flop_effective_hand_strength_two_one_and_flush_draw,
    **flop_effective_hand_strength_two_one_no_flush_draw,
    **flop_effective_hand_strength_ranibow_suited,
    **flop_effective_hand_strength_offsuit_mono_flush_draw_with_higher,
    **flop_effective_hand_strength_offsuit_mono_flush_draw_with_lower,
    **flop_effective_hand_strength_pairs_mono_flush_draw,
    **flop_effective_hand_strength_offsuit_mono_no_flush_draw,
    **flop_effective_hand_strength_pairs_mono_no_flush_draw,
    **flop_effective_hand_strengths_21_for_offsuit_cards,
    **flop_effective_hand_strengths_21_for_pairs,
    **flop_effective_hand_strengths_rainbow_for_offsuit_cards,
    **flop_effective_hand_strengths_rainbow_for_pairs
}

In [ ]:
#HERE
import json
with open("poker/ehs/postflop/ehs_postflop.json", 'w') as f:
    json.dump(effective_hand_strengths_postflop, f)

In [ ]:
effective_hand_strengths_postflop_dict = {
    "flop_effective_hand_strength_mono_and_flush": flop_effective_hand_strength_mono_and_flush,
    "flop_effective_hand_strength_mono_and_no_flush": flop_effective_hand_strength_mono_and_no_flush,
    "flop_effective_hand_strength_two_one_and_flush_draw": flop_effective_hand_strength_two_one_and_flush_draw,
    "flop_effective_hand_strength_two_one_no_flush_draw": flop_effective_hand_strength_two_one_no_flush_draw,
    "flop_effective_hand_strength_ranibow_suited": flop_effective_hand_strength_ranibow_suited,
    "flop_effective_hand_strength_offsuit_mono_flush_draw_with_higher": flop_effective_hand_strength_offsuit_mono_flush_draw_with_higher,
    "flop_effective_hand_strength_offsuit_mono_flush_draw_with_lower": flop_effective_hand_strength_offsuit_mono_flush_draw_with_lower,
    "flop_effective_hand_strength_pairs_mono_flush_draw": flop_effective_hand_strength_pairs_mono_flush_draw,
    "flop_effective_hand_strength_offsuit_mono_no_flush_draw": flop_effective_hand_strength_offsuit_mono_no_flush_draw,
    "flop_effective_hand_strength_pairs_mono_no_flush_draw": flop_effective_hand_strength_pairs_mono_no_flush_draw,
    "flop_effective_hand_strengths_21_for_offsuit_cards": flop_effective_hand_strengths_21_for_offsuit_cards,
    "flop_effective_hand_strengths_21_for_pairs": flop_effective_hand_strengths_21_for_pairs, 
    "flop_effective_hand_strengths_rainbow_for_offsuit_cards": flop_effective_hand_strengths_rainbow_for_offsuit_cards,
    "flop_effective_hand_strengths_rainbow_for_pairs": flop_effective_hand_strengths_rainbow_for_pairs
    }

In [ ]:
for k, v in effective_hand_strengths_postflop_dict.items():
    with open(f"poker/ehs/postflop/{k}.json", 'w') as f:
        json.dump(v, f)

In [ ]:
len(effective_hand_strengths_postflop)

279994

In [ ]:
#for k,v in sorted(effective_hand_strengths_postflop.items(), key=lambda item: item[1], reverse=True):
#    print(k, v)

In [ ]:
def compute_mono_flopturn_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    suited = "s"
    first_suit = "c"
    type = "_mono_"
    if made_flush:
        flush = "f"
        second_suit = first_suit
    else:
        flush = "n"
        second_suit = "s"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 4):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + second_suit)
            wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            deck.return_cards(flop)
            deck.return_cards(turn)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[1].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                 +str(combination[3].value)
                + type
                + flush
            ] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_mono_flopturn_for_suited_cards_with_flush = compute_mono_flopturn_effective_hand_strengths_for_suited_cards(200)
effective_hand_strengths_mono_flopturn_for_suited_cards_no_flush = compute_mono_flopturn_effective_hand_strengths_for_suited_cards(200, False)
#for k,v in effective_hand_strengths_mono_flopturn_for_suited_cards_no_flush.items():
#    print(k, v)

In [ ]:
def compute_31_flopturn_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    suited = "s"
    first_suit = "c"
    if made_flush:
        flush = "f"
        second_suit = first_suit
        third_suit = "s"
    else:
        flush = "n"
        second_suit = "s"
        third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 3):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + second_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + second_suit)
            third_card = deck.draw_card_by_tag(str(combination[2].value) + second_suit)
            third_suit_rank_list = [
                combination[0],
                combination[1],
                combination[2]
            ] + create_a_list_of_lower_ranks(combination[-1])[::-1]
            if made_flush:
                if starting_hand[1] > combination[-1]:
                    third_suit_rank_list = [starting_hand[0], starting_hand[1]] + third_suit_rank_list
                elif starting_hand[0] > combination[-1]:
                    third_suit_rank_list = [starting_hand[0]] + third_suit_rank_list
            for rank in third_suit_rank_list:
                flop = np.concatenate(
                    (
                       first_card,
                       second_card,
                       third_card
                    )
                )
                turn = deck.draw_card_by_tag(str(rank.value) + third_suit)
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(turn)
                board = sort_hand(np.concatenate((flop, turn)))
                effective_hand_strengths[
                    str(starting_hand[0].value)
                    + str(starting_hand[1].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                     +str(board[3].rank.value)
                    + "_threeone_"
                    + flush
                ] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations**2
            deck.return_cards(flop)
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_31_flopturn_for_suited_cards_with_flush = compute_31_flopturn_effective_hand_strengths_for_suited_cards(200, True)
effective_hand_strengths_31_flopturn_for_suited_cards_no_flush = compute_31_flopturn_effective_hand_strengths_for_suited_cards(200, False)
#for k,v in effective_hand_strengths_31_flopturn_for_suited_cards_with_flush.items():
#    print(k, v)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_suited_cards_no_flush.items():
#    print(k, v)

In [ ]:
def compute_22_flopturn_effective_hand_strengths_for_suited_cards(
    iterations: int, flush_draw=True
):
    first_suit = "c"
    suited = "s"
    iterations_sq = iterations**2
    if flush_draw:
        flush = "d"
        second_suit = first_suit
        third_suit = "s"
    else:
        flush = "n"
        second_suit = "s"
        third_suit = "d"

    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if flush_draw:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + second_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + second_suit)
            third_suit_rank_list = [
                combination[0]
            ] + create_a_list_of_lower_ranks(combination[0])[::-1]
            if flush_draw:
                if starting_hand[1] > combination[0]:
                    third_suit_rank_list = [starting_hand[0], starting_hand[1]] + third_suit_rank_list
                elif starting_hand[0] > combination[0]:
                    third_suit_rank_list = [starting_hand[0]] + third_suit_rank_list
            for ranks in itertools.combinations(third_suit_rank_list, 2):
                third_card = deck.draw_card_by_tag(str(ranks[0].value) + third_suit)
                flop = np.concatenate(
                    (
                        first_card,
                        second_card,
                        third_card,
                    )
                )
                turn = deck.draw_card_by_tag(str(ranks[1].value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn)))
                tag =  str(starting_hand[0].value) + str(starting_hand[1].value) + suited + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + "_twotwo_" + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(third_card)
                    deck.return_cards(turn)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                
                deck.return_cards(third_card)
                deck.return_cards(turn)
                effective_hand_strengths[tag] = (
                    wins * (iterations - negative_potential)
                    + (iterations - wins) * positive_potential
                ) / iterations_sq
            deck.return_cards(first_card)        
            deck.return_cards(second_card)       
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_22_flopturn_for_suited_cards_with_flush_draw = compute_22_flopturn_effective_hand_strengths_for_suited_cards(200, True)
effective_hand_strengths_22_flopturn_for_suited_cards_no_flush_draw = compute_22_flopturn_effective_hand_strengths_for_suited_cards(200, False)

In [ ]:
#for k,v in effective_hand_strengths_22_flopturn_for_suited_cards_with_flush_draw.items():
#    print(k, v)

In [ ]:
#for k,v in effective_hand_strengths_22_flopturn_for_suited_cards_no_flush_draw.items():
#    print(k, v)

In [ ]:
def compute_twooneone_flopturn_effective_hand_strengths_for_suited_cards(
    iterations: int, flush_draw=True
):
    first_suit = "c"
    suited = "s"
    iterations_sq = iterations**2
    if flush_draw:
        flush = "d"
        second_suit = first_suit
        third_suit = "s"
        fourth_suit = "d"
    else:
        flush = "n"
        second_suit = "s"
        third_suit = "d"
        fourth_suit = "h"

    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if flush_draw:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + second_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + second_suit)
            third_suit_rank_list = [combination[0]] + create_a_list_of_lower_ranks(
                combination[0]
            )[::-1]
            if flush_draw:
                if starting_hand[1] > combination[0]:
                    third_suit_rank_list = [
                        starting_hand[0],
                        starting_hand[1],
                    ] + third_suit_rank_list
                elif starting_hand[0] > combination[0]:
                    third_suit_rank_list = [starting_hand[0]] + third_suit_rank_list
            for ranks in itertools.combinations(third_suit_rank_list, 1):
                third_card = deck.draw_card_by_tag(str(ranks[0].value) + third_suit)
                flop = np.concatenate(
                    (
                        first_card,
                        second_card,
                        third_card,
                    )
                )
                fourth_suit_rank_list = [ranks[0]] + create_a_list_of_lower_ranks(
                    ranks[0]
                )
                for fourth_rank in fourth_suit_rank_list:
                    turn = deck.draw_card_by_tag(str(fourth_rank.value) + fourth_suit)
                    board = sort_hand(np.concatenate((flop, turn)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[1].value)
                        + suited
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + "_twooneone_"
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(turn)
                        continue
                    wins, positive_potential, negative_potential = (
                        perform_n_postturn_iterations(
                            iterations, deck, cards, flop, turn
                        )
                    )

                    deck.return_cards(turn)
                    effective_hand_strengths[tag] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations_sq
                deck.return_cards(third_card)
            deck.return_cards(first_card)
            deck.return_cards(second_card)
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_211_flopturn_for_suited_cards_with_flush_draw = compute_twooneone_flopturn_effective_hand_strengths_for_suited_cards(200)
effective_hand_strengths_211_flopturn_for_suited_cards_no_flush_draw = compute_twooneone_flopturn_effective_hand_strengths_for_suited_cards(200, False)

In [ ]:
#for k,v in effective_hand_strengths_211_flopturn_for_suited_cards_with_flush_draw.items():
#    print(k,v)

In [ ]:
#for k,v in effective_hand_strengths_211_flopturn_for_suited_cards_no_flush_draw.items():
#    print(k,v)

In [ ]:
def compute_rainbow_flopturn_effective_hand_strengths_for_suited_cards(iterations: int):
    suited = "s"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations_with_replacement(rank_list[::-1], 4):
            counter = Counter(combination)
            if counter.most_common(1)[0][1] == 4:
                if (
                    combination[0] == starting_hand[0]
                    or combination[3] == starting_hand[1]
                ):
                    continue
            elif counter.most_common(1)[0][1] == 3:
                if (
                    combination[0] == starting_hand[0]
                    and combination[3] == starting_hand[1]
                ):
                    continue
            elif (
                counter.most_common(2)[1][1] == 2
                and combination[0] == starting_hand[0]
                and combination[3] == starting_hand[1]
            ):
                continue
            if (
                combination[0] == starting_hand[0]
                and combination[3] == starting_hand[1]
            ):
                if combination[1] == starting_hand[0]:
                    temp_first_suit = second_suit
                    temp_second_suit = third_suit
                    temp_third_suit = first_suit
                    temp_fourth_suit = fourth_suit
                else:
                    temp_first_suit = second_suit
                    temp_second_suit = first_suit
                    temp_third_suit = third_suit
                    temp_fourth_suit = fourth_suit
            elif (
                combination[0] == starting_hand[0] or combination[0] == starting_hand[1]
            ):
                temp_first_suit = fourth_suit
                temp_second_suit = second_suit
                temp_third_suit = third_suit
                temp_fourth_suit = first_suit
            else:
                temp_first_suit = first_suit
                temp_second_suit = second_suit
                temp_third_suit = third_suit
                temp_fourth_suit = fourth_suit

            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + temp_first_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + temp_second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + temp_third_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + temp_fourth_suit)
            wins, positive_potential, negative_potential = (
                perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            )
            deck.return_cards(flop)
            deck.return_cards(turn)
            board = sort_hand(np.concatenate((flop, turn)))
            tag = (
                str(starting_hand[0].value)
                + str(starting_hand[1].value)
                + suited
                + "_"
                + str(board[0].rank.value)
                + str(board[1].rank.value)
                + str(board[2].rank.value)
                + str(board[3].rank.value)
                + type
                + flush
            )
            effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations_sq
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_rainbow_flopturn_for_suited_cards = compute_rainbow_flopturn_effective_hand_strengths_for_suited_cards(200)

In [ ]:
#for k,v in effective_hand_strengths_rainbow_flopturn_for_suited_cards.items():
#    print(k,v)

In [ ]:
def compute_mono_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush(iterations: int, with_higher_card = True):
    name = "o"
    type = "_mono_"
    flush = "f_higher" if with_higher_card else "f_lower" 
    first_suit = "c"
    second_suit = "s"
    flush_suit = first_suit if with_higher_card else second_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        if with_higher_card:
            rank_list.remove(starting_hand[0])
        else:
            rank_list.remove(starting_hand[1])
        possible_combinations = itertools.combinations(rank_list[::-1], 4)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            deck.return_cards(flop)
            deck.return_cards(turn)
            tag =  str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + str(turn[0].rank.value) + type + flush
            effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_higher = compute_mono_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush(200, True)
effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_lower = compute_mono_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush(200, False)

In [ ]:
#for k,v in effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_higher.items():
#    print(k, v)

In [ ]:
#for k,v in effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_lower.items():
#    print(k, v)

In [ ]:
def compute_mono_flopturn_effective_hand_strengths_for_pairs_with_flush(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "flush" 
    first_suit = "c"
    second_suit = "s"
    flush_suit = first_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        rank_list.remove(starting_hand[0])
        possible_combinations = itertools.combinations(rank_list[::-1], 4)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            deck.return_cards(flop)
            deck.return_cards(turn)
            tag =  str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + str(turn[0].rank.value) + type + flush
            effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_mono_flopturn_for_pairs_with_flush = compute_mono_flopturn_effective_hand_strengths_for_pairs_with_flush(200)

In [ ]:
#for k,v in effective_hand_strengths_mono_flopturn_for_pairs_with_flush.items():
#    print(k, v)

In [ ]:
def compute_mono_flopturn_effective_hand_strengths_for_offsuit_cards_no_flush(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    flush_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        possible_combinations = itertools.combinations(rank_list[::-1], 4)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            deck.return_cards(flop)
            deck.return_cards(turn)
            tag =  str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + str(turn[0].rank.value) + type + flush
            effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_mono_flopturn_for_offsuit_cards_no_flush = compute_mono_flopturn_effective_hand_strengths_for_offsuit_cards_no_flush(200)
#for k,v in effective_hand_strengths_mono_flopturn_for_offsuit_cards_no_flush.items():
#    print(k, v)

In [ ]:
def compute_mono_flopturn_effective_hand_strengths_for_pairs_no_flush(iterations: int):
    name = "o"
    type = "_mono_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    flush_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        possible_combinations = itertools.combinations(rank_list[::-1], 4)
        for combination in possible_combinations:
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
            deck.return_cards(flop)
            deck.return_cards(turn)
            tag =  str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(flop[0].rank.value) + str(flop[1].rank.value) + str(flop[2].rank.value) + str(turn[0].rank.value) + type + flush
            effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_mono_flopturn_for_pairs_no_flush = compute_mono_flopturn_effective_hand_strengths_for_pairs_no_flush(200)
#for k,v in effective_hand_strengths_mono_flopturn_for_pairs_no_flush.items():
#    print(k, v)

In [ ]:
def compute_threeone_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush_draw(iterations: int, with_higher_card = True):
    name = "o"
    type = "_threeone_"
    flush = "d_higher" if with_higher_card else "d_lower" 
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    flush_suit = first_suit if with_higher_card else second_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        if with_higher_card:
            rank_list.remove(starting_hand[0])
        else:
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            for fourth_card in list(Rank)[::-1]:
                turn = deck.draw_card_by_tag(str(fourth_card.value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(turn)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(flop)
            
    return effective_hand_strengths

In [ ]:
def compute_threeone_flopturn_effective_hand_strengths_for_pairs_with_flush_draw(iterations: int):
    name = "o"
    type = "_threeone_"
    flush = "d"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    flush_suit = first_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        rank_list.remove(starting_hand[0])
        for combination in itertools.combinations(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            for fourth_card in list(Rank)[::-1]:
                turn = deck.draw_card_by_tag(str(fourth_card.value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(turn)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(flop)
            
    return effective_hand_strengths

In [ ]:
#TODO Look at the outputs
effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_higher = compute_threeone_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush_draw(200, True)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_higher.items():
#    print(k, v)

In [ ]:
effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_lower = compute_threeone_flopturn_effective_hand_strengths_for_offsuit_cards_with_flush_draw(200, False)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_lower.items():
#    print(k,v)

In [ ]:
effective_hand_strengths_31_flopturn_for_pairs_flush_draw = compute_threeone_flopturn_effective_hand_strengths_for_pairs_with_flush_draw(200)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_pairs_flush_draw.items():
#    print(k, v)

In [ ]:
def compute_threeone_flopturn_effective_hand_strengths_for_offsuit_cards_no_flush_draw(iterations: int):
    name = "o"
    type = "_threeone_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + third_suit),
                )
            )
            for fourth_card in list(Rank)[::-1]:
                turn = deck.draw_card_by_tag(str(fourth_card.value) + fourth_suit)
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(turn)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(flop)
            
    return effective_hand_strengths

In [ ]:
def compute_threeone_flopturn_effective_hand_strengths_for_pairs_no_flush_draw(iterations: int):
    name = "o"
    type = "_threeone_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + third_suit),
                )
            )
            for fourth_card in list(Rank)[::-1]:
                turn = deck.draw_card_by_tag(str(fourth_card.value) + fourth_suit)
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(turn)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(flop)
            
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_31_flopturn_for_offsuit_cards_no_draw = compute_threeone_flopturn_effective_hand_strengths_for_offsuit_cards_no_flush_draw(200)
effective_hand_strengths_31_flopturn_for_pairs_no_draw = compute_threeone_flopturn_effective_hand_strengths_for_pairs_no_flush_draw(200)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_offsuit_cards_no_draw.items():
#    print(k, v)

In [ ]:
#for k,v in effective_hand_strengths_31_flopturn_for_pairs_no_draw.items():
#    print(k, v)

In [ ]:
def compute_twotwo_flopturn_effective_hand_strengths_for_offsuit_cards(iterations: int):
    name = "o"
    type = "_twotwo_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for ranks in itertools.combinations(rank_list[::-1], 2):
                third_card = deck.draw_card_by_tag(str(ranks[0].value) + fourth_suit)
                fourth_card = deck.draw_card_by_tag(str(ranks[1].value) + fourth_suit)
                flop = np.concatenate(
                (
                    first_card,
                    second_card,
                    third_card
                )
            )
                turn = fourth_card
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[1].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(third_card)
                    deck.return_cards(fourth_card)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(third_card)
                deck.return_cards(fourth_card)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(first_card)
            deck.return_cards(second_card)
            
    return effective_hand_strengths

In [ ]:
def compute_twotwo_flopturn_effective_hand_strengths_for_pairs(iterations: int):
    name = "o"
    type = "_twotwo_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for ranks in itertools.combinations(rank_list[::-1], 2):
                third_card = deck.draw_card_by_tag(str(ranks[0].value) + fourth_suit)
                fourth_card = deck.draw_card_by_tag(str(ranks[1].value) + fourth_suit)
                flop = np.concatenate(
                (
                    first_card,
                    second_card,
                    third_card
                )
            )
                turn = fourth_card
                board = sort_hand(np.concatenate((flop, turn)))
                tag = str(starting_hand[0].value) + str(starting_hand[0].value) + name + "_" + str(board[0].rank.value) + str(board[1].rank.value) + str(board[2].rank.value) + str(board[3].rank.value) + type + flush
                if tag in effective_hand_strengths:
                    deck.return_cards(third_card)
                    deck.return_cards(fourth_card)
                    continue
                wins, positive_potential, negative_potential = perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                deck.return_cards(third_card)
                deck.return_cards(fourth_card)
                effective_hand_strengths[tag] = (
                wins * (iterations - negative_potential)
                + (iterations - wins) * positive_potential
            ) / iterations**2
            deck.return_cards(first_card)
            deck.return_cards(second_card)
            
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_22_flopturn_for_offsuit_cards = compute_twotwo_flopturn_effective_hand_strengths_for_offsuit_cards(200)
#for k,v in effective_hand_strengths_22_flopturn_for_offsuit_cards.items():
#    print(k, v)

In [ ]:
effective_hand_strengths_22_flopturn_for_pairs = compute_twotwo_flopturn_effective_hand_strengths_for_pairs(200)

In [ ]:
#for k,v in effective_hand_strengths_22_flopturn_for_pairs.items():
#    print(k, v)

In [ ]:
def compute_twooneone_flopturn_effective_hand_strengths_for_offsuit_cards(
    iterations: int
):
    first_suit = "c"
    type = "_twooneone_"
    name = "o"
    flush = "n"
    iterations_sq = iterations**2
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"

    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for third_rank in itertools.combinations(rank_list[::-1], 1):
                third_card = deck.draw_card_by_tag(str(third_rank[0].value) + fourth_suit)
                for fourth_rank in itertools.combinations(rank_list[::-1], 1):
                    temp_suit = second_suit if fourth_rank[0] != starting_hand[1] else first_suit
                    fourth_card = deck.draw_card_by_tag(
                        str(fourth_rank[0].value) + temp_suit
                    )
                    flop = np.concatenate((first_card, second_card, third_card))
                    turn = fourth_card
                    board = sort_hand(np.concatenate((flop, turn)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[1].value)
                        + name
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + type
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(fourth_card)
                        continue
                    wins, positive_potential, negative_potential = (
                        perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                    )
                    deck.return_cards(fourth_card)
                    effective_hand_strengths[tag] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations_sq
                deck.return_cards(third_card)
            deck.return_cards(first_card)
            deck.return_cards(second_card)

    return effective_hand_strengths

In [ ]:
effective_hand_strengths_211_flopturn_for_offsuit_cards = compute_twooneone_flopturn_effective_hand_strengths_for_offsuit_cards(200)
#for k,v in effective_hand_strengths_211_flopturn_for_offsuit_cards.items():
#    print(k, v)

In [ ]:
def compute_twooneone_flopturn_effective_hand_strengths_for_pairs(
    iterations: int
):
    first_suit = "c"
    type = "_twooneone_"
    name = "o"
    flush = "n"
    iterations_sq = iterations**2
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"

    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for third_rank in itertools.combinations(rank_list[::-1], 1):
                third_card = deck.draw_card_by_tag(str(third_rank[0].value) + fourth_suit)
                fourth_rank_list = list(Rank)
                fourth_rank_list.remove(starting_hand[0])
                for fourth_rank in itertools.combinations(fourth_rank_list[::-1], 1):
                    temp_suit = second_suit
                    fourth_card = deck.draw_card_by_tag(
                        str(fourth_rank[0].value) + temp_suit
                    )
                    flop = np.concatenate((first_card, second_card, third_card))
                    turn = fourth_card
                    board = sort_hand(np.concatenate((flop, turn)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[0].value)
                        + name
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + type
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(fourth_card)
                        continue
                    wins, positive_potential, negative_potential = (
                        perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                    )
                    deck.return_cards(fourth_card)
                    effective_hand_strengths[tag] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations_sq
                deck.return_cards(third_card)
            deck.return_cards(first_card)
            deck.return_cards(second_card)

    return effective_hand_strengths

In [ ]:
effective_hand_strengths_211_flopturn_for_pairs = compute_twooneone_flopturn_effective_hand_strengths_for_pairs(200)
#for k,v in effective_hand_strengths_211_flopturn_for_pairs.items():
#    print(k, v)

In [ ]:
def compute_rainbow_flopturn_effective_hand_strengths_for_offsuit_cards(iterations: int):
    suited = "o"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        first_rank_list = list(Rank)
        first_rank_list.remove(starting_hand[0])
        second_rank_list = list(Rank)
        second_rank_list.remove(starting_hand[1])
        third_rank_list = list(Rank)
        
        for first_combination in itertools.combinations(first_rank_list[::-1], 1):
            first_card =  deck.draw_card_by_tag(str(first_combination[0].value) + first_suit)
            for second_combination in itertools.combinations(second_rank_list[::-1], 1):
                second_card = deck.draw_card_by_tag(str(second_combination[0].value) + second_suit)
                for third_combination in itertools.combinations_with_replacement(third_rank_list[::-1], 2):
                    third_card =  deck.draw_card_by_tag(str(third_combination[0].value) + third_suit)
                    fourth_card =  deck.draw_card_by_tag(str(third_combination[1].value) + fourth_suit)
                    flop = np.concatenate(
                        (
                           first_card,
                           second_card,
                           third_card
                        )
                    )
                    turn = fourth_card
                    board = sort_hand(np.concatenate((flop, turn)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[1].value)
                        + suited
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + type
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(third_card)
                        deck.return_cards(fourth_card)
                        continue
                    wins, positive_potential, negative_potential = (
                        perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                    )
                    deck.return_cards(third_card)
                    deck.return_cards(fourth_card)
                    board = sort_hand(np.concatenate((flop, turn)))
                    effective_hand_strengths[tag] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations_sq
                deck.return_cards(second_card)
            deck.return_cards(first_card)
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_rainbow_flopturn_for_offsuit_cards = compute_rainbow_flopturn_effective_hand_strengths_for_offsuit_cards(200)
#for k,v in effective_hand_strengths_rainbow_flopturn_for_offsuit_cards.items():
#    print(k, v)

In [ ]:
def compute_rainbow_flopturn_effective_hand_strengths_for_pairs(iterations: int):
    suited = "o"
    type = "_rainbow_"
    flush = "n"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    iterations_sq = iterations**2
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        first_rank_list = list(Rank)
        first_rank_list.remove(starting_hand[0])
        second_rank_list = list(Rank)
        second_rank_list.remove(starting_hand[0])
        third_rank_list = list(Rank)
        
        for first_combination in itertools.combinations(first_rank_list[::-1], 1):
            first_card =  deck.draw_card_by_tag(str(first_combination[0].value) + first_suit)
            for second_combination in itertools.combinations(second_rank_list[::-1], 1):
                second_card = deck.draw_card_by_tag(str(second_combination[0].value) + second_suit)
                for third_combination in itertools.combinations_with_replacement(third_rank_list[::-1], 2):
                    third_card =  deck.draw_card_by_tag(str(third_combination[0].value) + third_suit)
                    fourth_card =  deck.draw_card_by_tag(str(third_combination[1].value) + fourth_suit)
                    flop = np.concatenate(
                        (
                           first_card,
                           second_card,
                           third_card
                        )
                    )
                    turn = fourth_card
                    board = sort_hand(np.concatenate((flop, turn)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[0].value)
                        + suited
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + type
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(third_card)
                        deck.return_cards(fourth_card)
                        continue
                    wins, positive_potential, negative_potential = (
                        perform_n_postturn_iterations(iterations, deck, cards, flop, turn)
                    )
                    deck.return_cards(third_card)
                    deck.return_cards(fourth_card)
                    board = sort_hand(np.concatenate((flop, turn)))
                    effective_hand_strengths[tag] = (
                        wins * (iterations - negative_potential)
                        + (iterations - wins) * positive_potential
                    ) / iterations_sq
                deck.return_cards(second_card)
            deck.return_cards(first_card)
    return effective_hand_strengths

In [ ]:
effective_hand_strengths_rainbow_flopturn_for_pairs = compute_rainbow_flopturn_effective_hand_strengths_for_pairs(200)
#for k,v in effective_hand_strengths_rainbow_flopturn_for_pairs.items():
#    print(k, v)

In [ ]:
#TODO CREATE DICT AND JSON

In [ ]:
effective_hand_strengths_postturn = {
    **effective_hand_strengths_mono_flopturn_for_suited_cards_with_flush,
    **effective_hand_strengths_mono_flopturn_for_suited_cards_no_flush,
    **effective_hand_strengths_31_flopturn_for_suited_cards_with_flush,
    **effective_hand_strengths_31_flopturn_for_suited_cards_no_flush,
    **effective_hand_strengths_22_flopturn_for_suited_cards_with_flush_draw,
    **effective_hand_strengths_22_flopturn_for_suited_cards_no_flush_draw,
    **effective_hand_strengths_211_flopturn_for_suited_cards_with_flush_draw,
    **effective_hand_strengths_211_flopturn_for_suited_cards_no_flush_draw,
    **effective_hand_strengths_rainbow_flopturn_for_suited_cards,
    **effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_higher,
    **effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_lower,
    **effective_hand_strengths_mono_flopturn_for_offsuit_cards_no_flush,
    **effective_hand_strengths_mono_flopturn_for_pairs_with_flush,
    **effective_hand_strengths_mono_flopturn_for_pairs_no_flush,
    **effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_higher,
    **effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_lower,
    **effective_hand_strengths_31_flopturn_for_pairs_flush_draw,
    **effective_hand_strengths_31_flopturn_for_offsuit_cards_no_draw,
    **effective_hand_strengths_31_flopturn_for_pairs_no_draw,
    **effective_hand_strengths_22_flopturn_for_offsuit_cards,
    **effective_hand_strengths_22_flopturn_for_pairs,
    **effective_hand_strengths_211_flopturn_for_offsuit_cards,
    **effective_hand_strengths_211_flopturn_for_pairs,
    **effective_hand_strengths_rainbow_flopturn_for_offsuit_cards,
    **effective_hand_strengths_rainbow_flopturn_for_pairs
}

In [ ]:
with open("poker/ehs/postturn/ehs_postturn.json", 'w') as f:
    json.dump(effective_hand_strengths_postturn, f)

In [ ]:
effective_hand_strengths_postturn_dict = {
"effective_hand_strengths_mono_flopturn_for_suited_cards_with_flush": effective_hand_strengths_mono_flopturn_for_suited_cards_with_flush,
"effective_hand_strengths_mono_flopturn_for_suited_cards_no_flush": effective_hand_strengths_mono_flopturn_for_suited_cards_no_flush, 
"effective_hand_strengths_31_flopturn_for_suited_cards_with_flush": effective_hand_strengths_31_flopturn_for_suited_cards_with_flush, 
"effective_hand_strengths_31_flopturn_for_suited_cards_no_flush": effective_hand_strengths_31_flopturn_for_suited_cards_no_flush, 
"effective_hand_strengths_22_flopturn_for_suited_cards_with_flush_draw": effective_hand_strengths_22_flopturn_for_suited_cards_with_flush_draw, 
"effective_hand_strengths_22_flopturn_for_suited_cards_no_flush_draw": effective_hand_strengths_22_flopturn_for_suited_cards_no_flush_draw, 
"effective_hand_strengths_211_flopturn_for_suited_cards_with_flush_draw": effective_hand_strengths_211_flopturn_for_suited_cards_with_flush_draw, 
"effective_hand_strengths_211_flopturn_for_suited_cards_no_flush_draw": effective_hand_strengths_211_flopturn_for_suited_cards_no_flush_draw, 
"effective_hand_strengths_rainbow_flopturn_for_suited_cards": effective_hand_strengths_rainbow_flopturn_for_suited_cards, 
"effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_higher": effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_higher, 
"effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_lower": effective_hand_strengths_mono_flopturn_for_offsuit_cards_flush_with_lower, 
"effective_hand_strengths_mono_flopturn_for_offsuit_cards_no_flush": effective_hand_strengths_mono_flopturn_for_offsuit_cards_no_flush, 
"effective_hand_strengths_mono_flopturn_for_pairs_with_flush": effective_hand_strengths_mono_flopturn_for_pairs_with_flush, 
"effective_hand_strengths_mono_flopturn_for_pairs_no_flush": effective_hand_strengths_mono_flopturn_for_pairs_no_flush, 
"effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_higher": effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_higher, 
"effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_lower": effective_hand_strengths_31_flopturn_for_offsuit_cards_flush_draw_with_lower, 
"effective_hand_strengths_31_flopturn_for_pairs_flush_draw": effective_hand_strengths_31_flopturn_for_pairs_flush_draw, 
"effective_hand_strengths_31_flopturn_for_offsuit_cards_no_draw": effective_hand_strengths_31_flopturn_for_offsuit_cards_no_draw, 
"effective_hand_strengths_31_flopturn_for_pairs_no_draw": effective_hand_strengths_31_flopturn_for_pairs_no_draw, 
"effective_hand_strengths_22_flopturn_for_offsuit_cards": effective_hand_strengths_22_flopturn_for_offsuit_cards, 
"effective_hand_strengths_22_flopturn_for_pairs": effective_hand_strengths_22_flopturn_for_pairs,
"effective_hand_strengths_211_flopturn_for_offsuit_cards": effective_hand_strengths_211_flopturn_for_offsuit_cards, 
"effective_hand_strengths_211_flopturn_for_pairs": effective_hand_strengths_211_flopturn_for_pairs, 
"effective_hand_strengths_rainbow_flopturn_for_offsuit_cards": effective_hand_strengths_rainbow_flopturn_for_offsuit_cards, 
"effective_hand_strengths_rainbow_flopturn_for_pairs": effective_hand_strengths_rainbow_flopturn_for_pairs 
    }

In [ ]:
for k, v in effective_hand_strengths_postturn_dict.items():
    with open(f"poker/ehs/postturn/{k}.json", 'w') as f:
        json.dump(v, f)

In [ ]:
#NORMALIZATION
with open("poker/ehs/postturn/ehs_postturn.json", 'r') as f:
    my_json = json.load(f)

In [ ]:
max_val = max(my_json.values())
min_val = min(my_json.values())
for k,v in my_json.items():
    my_json[k] = (v - min_val) / (max_val - min_val)

In [ ]:
#for k,v in my_json.items():
#    print(k, v)

In [ ]:
def compute_mono_flopturnriver_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    type = "_mono_"
    suited = "s"
    first_suit = "c"
    if made_flush:
        flush = "f"
        second_suit = first_suit
    else:
        flush = "n"
        second_suit = "s"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 5):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + second_suit)
            river = deck.draw_card_by_tag(str(combination[4].value) + second_suit)
            win_rate = perform_n_postriver_iterations(iterations, deck, cards, flop, turn, river)
            deck.return_cards(flop)
            deck.return_cards(turn)
            deck.return_cards(river)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[1].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                + str(combination[3].value)
                + str(combination[4].value) 
                + type
                + flush
            ] = win_rate
    return effective_hand_strengths

In [ ]:
mono_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush = compute_mono_flopturnriver_effective_hand_strengths_for_suited_cards(100, True)

In [ ]:
mono_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush = compute_mono_flopturnriver_effective_hand_strengths_for_suited_cards(100, False)

In [ ]:
def compute_fourone_flopturnriver_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    type = "_fourone_"
    suited = "s"
    first_suit = "c"
    if made_flush:
        flush = "f"
        second_suit = first_suit
    else:
        flush = "n"
        second_suit = "s"
    third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 4):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + second_suit)
            if made_flush:
                if starting_hand[0] > combination[0]:
                    flush = f"f_{starting_hand[0].value}_high"
                else:
                    flush = f"f_{combination[0].value}_high"
            for fifth_rank in itertools.combinations(list(Rank)[::-1], 1):
                river = deck.draw_card_by_tag(str(fifth_rank[0].value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[1].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                deck.return_cards(river)
                effective_hand_strengths[tag] = win_rate
            deck.return_cards(flop)
            deck.return_cards(turn)
    return effective_hand_strengths

In [ ]:
fourone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush = compute_fourone_flopturnriver_effective_hand_strengths_for_suited_cards(100, True)


In [ ]:
fourone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush = compute_fourone_flopturnriver_effective_hand_strengths_for_suited_cards(100, False)


In [ ]:
#for k, v in fourone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush.items():
#    print(k, v)

In [ ]:
def compute_threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards(
    iterations: int, made_flush=True
):
    type = "_threeoneone_"
    suited = "s"
    first_suit = "c"
    if made_flush:
        flush = "f"
        second_suit = first_suit
    else:
        flush = "n"
        second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        rank_list = list(Rank)
        if made_flush:
            rank_list.remove(starting_hand[0])
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 3):
            if made_flush:
                if starting_hand[0] > combination[0]:
                    flush = f"f_{starting_hand[0].value}_high"
                else:
                    flush = f"f_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + second_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + second_suit),
                )
            )
            for ranks in itertools.combinations_with_replacement(list(Rank)[::-1], 2):
                turn = deck.draw_card_by_tag(str(ranks[0].value) + third_suit)
                river = deck.draw_card_by_tag(str(ranks[1].value) + fourth_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[1].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                deck.return_cards(turn)
                deck.return_cards(river)
                effective_hand_strengths[tag] = win_rate
            deck.return_cards(flop)
    return effective_hand_strengths

In [ ]:
threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush = compute_threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards(100)
threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush = compute_threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards(100, False)

In [ ]:
def compute_twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards(
    iterations: int
):
    type = "_twooneoneone_"
    flush = "n"
    suited = "s"
    first_suit = "c"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + first_suit),
            )
        )
        first_ranks = list(Rank)
        first_ranks.remove(starting_hand[0])
        first_ranks.remove(starting_hand[1])
        for rank in itertools.combinations(first_ranks[::-1], 1):
            rank_list = list(Rank)
            first_card = deck.draw_card_by_tag(str(rank[0].value) + first_suit)
            for combination in itertools.combinations(rank_list[::-1], 2):
                second_card = deck.draw_card_by_tag(str(combination[0].value) + second_suit) 
                third_card = deck.draw_card_by_tag(str(combination[1].value) + second_suit) 
                flop = np.concatenate(
                        (
                            first_card,
                            second_card,
                            third_card
                        )
                    )
                for ranks in itertools.combinations_with_replacement(rank_list[::-1], 2):
                    turn = deck.draw_card_by_tag(str(ranks[0].value) + third_suit)
                    river = deck.draw_card_by_tag(str(ranks[1].value) + fourth_suit)
                    board = sort_hand(np.concatenate((flop, turn, river)))
                    tag = (
                            str(starting_hand[0].value)
                            + str(starting_hand[1].value)
                            + suited
                            + "_"
                            + str(board[0].rank.value)
                            + str(board[1].rank.value)
                            + str(board[2].rank.value)
                            + str(board[3].rank.value)
                            + str(board[4].rank.value)
                            + type
                            + flush
                        )
                    if tag in effective_hand_strengths:
                        deck.return_cards(turn)
                        deck.return_cards(river)
                        continue
                    win_rate = perform_n_postriver_iterations(
                        iterations, deck, cards, flop, turn, river
                    )
                    deck.return_cards(turn)
                    deck.return_cards(river)
                    effective_hand_strengths[tag] = win_rate
                deck.return_cards(second_card)
                deck.return_cards(third_card)
            deck.return_cards(first_card)
    return effective_hand_strengths

In [ ]:
twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards = compute_twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards(100)
#for k,v in t.items():
#    print(k ,v)

In [ ]:
def compute_mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(
    iterations: int, with_higher_card=True
):
    type = "_mono_"
    
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    if with_higher_card:
        flush_suit = first_suit
        flush = "f_higher"
    else:
        flush_suit = second_suit
        flush = "f_lower"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        if with_higher_card:
            rank_list.remove(starting_hand[0])
        else:
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 5):
            if with_higher_card:
                if starting_hand[0] > combination[0]:
                    flush = f"f_higher_{starting_hand[0].value}_high"
                else:
                    flush = f"f_higher_{combination[0].value}_high"
            else:
                if starting_hand[1] > combination[0]:
                    flush = f"f_lower_{starting_hand[1].value}_high"
                else:
                    flush = f"f_lower_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            river = deck.draw_card_by_tag(str(combination[4].value) + flush_suit)
            win_rate = perform_n_postriver_iterations(iterations, deck, cards, flop, turn, river)
            deck.return_cards(flop)
            deck.return_cards(turn)
            deck.return_cards(river)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[1].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                + str(combination[3].value)
                + str(combination[4].value) 
                + type
                + flush
            ] = win_rate
    return effective_hand_strengths

In [ ]:
mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card = compute_mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(100, True)
mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card = compute_mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(100, False)

In [ ]:
def compute_mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush(
    iterations: int
):
    type = "_mono_"
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    flush = "f"
    flush_suit = first_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        rank_list.remove(starting_hand[0])
        for combination in itertools.combinations(rank_list[::-1], 5):
            if starting_hand[0] > combination[0]:
                flush = f"f_{starting_hand[0].value}_high"
            else:
                flush = f"f_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            river = deck.draw_card_by_tag(str(combination[4].value) + flush_suit)
            win_rate = perform_n_postriver_iterations(iterations, deck, cards, flop, turn, river)
            deck.return_cards(flop)
            deck.return_cards(turn)
            deck.return_cards(river)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[0].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                + str(combination[3].value)
                + str(combination[4].value) 
                + type
                + flush
            ] = win_rate
    return effective_hand_strengths

In [ ]:
def compute_mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush(
    iterations: int
):
    type = "_mono_"
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    flush = "n"
    flush_suit = first_suit
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 5):
            if starting_hand[0] > combination[0]:
                flush = f"f_{starting_hand[0].value}_high"
            else:
                flush = f"f_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            river = deck.draw_card_by_tag(str(combination[4].value) + flush_suit)
            win_rate = perform_n_postriver_iterations(iterations, deck, cards, flop, turn, river)
            deck.return_cards(flop)
            deck.return_cards(turn)
            deck.return_cards(river)
            effective_hand_strengths[
                str(starting_hand[0].value)
                + str(starting_hand[0].value)
                + suited
                + "_"
                + str(combination[0].value)
                + str(combination[1].value)
                + str(combination[2].value)
                + str(combination[3].value)
                + str(combination[4].value) 
                + type
                + flush
            ] = win_rate
    return effective_hand_strengths

In [ ]:
mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush = compute_mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush(100)

In [ ]:
mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush = compute_mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush(100)

In [ ]:
#for k,v in mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush.items():
#    print(k, v)

In [ ]:
def compute_fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(
    iterations: int, with_higher_card=True
):
    type = "_fourone_"
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    if with_higher_card:
        flush_suit = first_suit
        flush = "f_higher"
    else:
        flush_suit = second_suit
        flush = "f_lower"
    third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        if with_higher_card:
            rank_list.remove(starting_hand[0])
        else:
            rank_list.remove(starting_hand[1])
        for combination in itertools.combinations(rank_list[::-1], 4):
            if with_higher_card:
                if starting_hand[0] > combination[0]:
                    flush = f"f_higher_{starting_hand[0].value}_high"
                else:
                    flush = f"f_higher_{combination[0].value}_high"
            else:
                if starting_hand[1] > combination[0]:
                    flush = f"f_lower_{starting_hand[1].value}_high"
                else:
                    flush = f"f_lower_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            for last_rank in itertools.combinations(list(Rank)[::-1], 1):
                river = deck.draw_card_by_tag(str(last_rank[0].value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[1].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                effective_hand_strengths[tag] = win_rate
                deck.return_cards(river)
            deck.return_cards(flop)
            deck.return_cards(turn)
    return effective_hand_strengths

In [ ]:
fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card = compute_fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(100, True)
fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card = compute_fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush(100, False)

In [ ]:
#for k,v in fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card.items():
#    print(k,v)

In [ ]:
def compute_fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush(
    iterations: int
):
    type = "_fourone_"
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    flush = "f"
    flush_suit = "c"
    third_suit = "d"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        rank_list.remove(starting_hand[0])
        
        for combination in itertools.combinations(rank_list[::-1], 4):
            if starting_hand[0] > combination[0]:
                flush = f"f_{starting_hand[0].value}_high"
            else:
                flush = f"f_{combination[0].value}_high"
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            for last_rank in itertools.combinations(list(Rank)[::-1], 1):
                river = deck.draw_card_by_tag(str(last_rank[0].value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[0].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                effective_hand_strengths[tag] = win_rate
                deck.return_cards(river)
            deck.return_cards(flop)
            deck.return_cards(turn)
    return effective_hand_strengths

In [1]:
def compute_fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush(
    iterations: int
):
    type = "_fourone_"
    suited = "o"
    first_suit = "c"
    second_suit = "s"
    flush = "n"
    flush_suit = "d"
    third_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)     
        for combination in itertools.combinations(rank_list[::-1], 4):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + flush_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + flush_suit),
                )
            )
            turn = deck.draw_card_by_tag(str(combination[3].value) + flush_suit)
            for last_rank in itertools.combinations(list(Rank)[::-1], 1):
                river = deck.draw_card_by_tag(str(last_rank[0].value) + third_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[0].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                effective_hand_strengths[tag] = win_rate
                deck.return_cards(river)
            deck.return_cards(flop)
            deck.return_cards(turn)
    return effective_hand_strengths

In [ ]:
fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush = compute_fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush(100)
#for k,v in t.items():
#    print(k ,v)

In [11]:
fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush = compute_fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush(100)

In [ ]:
#HERE
def compute_threeoneone_flopturnriver_effective_hand_strengths_for_pairs(
    iterations: int
):
    type = "_threeoneone_"
    suited = "o"
    first_suit = "c"
    flush = "n"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 3):
            flop = np.concatenate(
                (
                    deck.draw_card_by_tag(str(combination[0].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[1].value) + third_suit),
                    deck.draw_card_by_tag(str(combination[2].value) + third_suit),
                )
            )
            for ranks in itertools.combinations_with_replacement(list(Rank)[::-1], 2):
                if ranks[1] == starting_hand[0] and ranks[0] == ranks[1]:
                    continue
                elif ranks[1] == starting_hand[0]:
                    temp_suit = fourth_suit
                else:
                    temp_suit = first_suit
                    
                turn = deck.draw_card_by_tag(str(ranks[0].value) + fourth_suit)
                river = deck.draw_card_by_tag(str(ranks[1].value) + temp_suit)
                board = sort_hand(np.concatenate((flop, turn, river)))
                tag = (
                    str(starting_hand[0].value)
                    + str(starting_hand[0].value)
                    + suited
                    + "_"
                    + str(board[0].rank.value)
                    + str(board[1].rank.value)
                    + str(board[2].rank.value)
                    + str(board[3].rank.value)
                    + str(board[4].rank.value)
                    + type
                    + flush
                )
                if tag in effective_hand_strengths:
                    deck.return_cards(turn)
                    deck.return_cards(river)
                    continue
                win_rate = perform_n_postriver_iterations(
                    iterations, deck, cards, flop, turn, river
                )
                deck.return_cards(turn)
                deck.return_cards(river)
                effective_hand_strengths[tag] = win_rate
            deck.return_cards(flop)
    return effective_hand_strengths

In [ ]:
threeoneone_flopturnriver_effective_hand_strengths_for_pairs = compute_threeoneone_flopturnriver_effective_hand_strengths_for_pairs(100)

In [ ]:
# HERE
def compute_twooneoneone_flopturnriver_effective_hand_strengths_for_pairs(
    iterations: int,
):
    type = "_twooneoneone_"
    suited = "o"
    first_suit = "c"
    flush = "n"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 1):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[0].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for ranks in itertools.combinations_with_replacement(rank_list[::-1], 2):
                if ranks[1] == starting_hand[0] and ranks[0] == ranks[1]:
                    continue
                elif ranks[1] == starting_hand[0]:
                    temp_suit = fourth_suit
                else:
                    temp_suit = first_suit
                third_card = deck.draw_card_by_tag(
                    str(ranks[0].value) + fourth_suit
                )
                flop = np.concatenate((first_card, second_card, third_card))
                turn = deck.draw_card_by_tag(str(ranks[1].value) + temp_suit)
                fifth_rank_list = list(Rank)
                fifth_rank_list.remove(starting_hand[0])
                for fifth_rank in itertools.combinations(fifth_rank_list[::-1], 1):
                    river = deck.draw_card_by_tag(str(fifth_rank[0].value) + second_suit)
                    board = sort_hand(np.concatenate((flop, turn, river)))
                    tag = (
                        str(starting_hand[0].value)
                        + str(starting_hand[0].value)
                        + suited
                        + "_"
                        + str(board[0].rank.value)
                        + str(board[1].rank.value)
                        + str(board[2].rank.value)
                        + str(board[3].rank.value)
                        + str(board[4].rank.value)
                        + type
                        + flush
                    )
                    if tag in effective_hand_strengths:
                        deck.return_cards(river)
                        continue
                    win_rate = perform_n_postriver_iterations(
                        iterations, deck, cards, flop, turn, river
                    )
                    deck.return_cards(river)
                    effective_hand_strengths[tag] = win_rate
                deck.return_cards(turn)
                deck.return_cards(third_card)
            deck.return_cards(second_card)
            deck.return_cards(first_card)
    return effective_hand_strengths

In [ ]:
# HERE
def compute_twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards(
    iterations: int,
):
    type = "_twooneoneone_"
    suited = "o"
    first_suit = "c"
    flush = "n"
    second_suit = "s"
    third_suit = "d"
    fourth_suit = "h"
    effective_hand_strengths = {}
    for starting_hand in itertools.combinations(list(Rank)[::-1], 2):
        deck = Deck()
        cards = np.concatenate(
            (
                deck.draw_card_by_tag(str(starting_hand[0].value) + first_suit),
                deck.draw_card_by_tag(str(starting_hand[1].value) + second_suit),
            )
        )
        rank_list = list(Rank)
        for combination in itertools.combinations(rank_list[::-1], 2):
            first_card = deck.draw_card_by_tag(str(combination[0].value) + third_suit)
            second_card = deck.draw_card_by_tag(str(combination[1].value) + third_suit)
            for third_card in itertools.combinations(rank_list[::-1], 1):
                third_card = deck.draw_card_by_tag(
                    str(third_card[0].value) + fourth_suit
                )
                flop = np.concatenate((first_card, second_card, third_card))
                fourth_card_list = list(Rank)
                fourth_card_list.remove(starting_hand[0])
                for fourth_card in itertools.combinations(fourth_card_list[::-1], 1):
                    turn = deck.draw_card_by_tag(str(fourth_card[0].value) + first_suit)
                    fifth_rank_list = list(Rank)
                    fifth_rank_list.remove(starting_hand[1])
                    for fifth_rank in itertools.combinations(fifth_rank_list[::-1], 1):
                        river = deck.draw_card_by_tag(str(fifth_rank[0].value) + second_suit)
                        board = sort_hand(np.concatenate((flop, turn, river)))
                        tag = (
                            str(starting_hand[0].value)
                            + str(starting_hand[1].value)
                            + suited
                            + "_"
                            + str(board[0].rank.value)
                            + str(board[1].rank.value)
                            + str(board[2].rank.value)
                            + str(board[3].rank.value)
                            + str(board[4].rank.value)
                            + type
                            + flush
                        )
                        if tag in effective_hand_strengths:
                            deck.return_cards(river)
                            continue
                        win_rate = perform_n_postriver_iterations(
                            iterations, deck, cards, flop, turn, river
                        )
                        deck.return_cards(river)
                        effective_hand_strengths[tag] = win_rate
                    deck.return_cards(turn)
                deck.return_cards(third_card)
            deck.return_cards(second_card)
            deck.return_cards(first_card)
    return effective_hand_strengths

In [ ]:
twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards = compute_twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards(100)

In [ ]:
twooneoneone_flopturnriver_effective_hand_strengths_for_pairs = compute_twooneoneone_flopturnriver_effective_hand_strengths_for_pairs(100)

In [ ]:
import json

In [ ]:
temp_dict = {
"mono_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush": mono_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush,
"mono_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush": mono_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush,
"mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card": mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card,
"mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card": mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card,
"mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush": mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush,
"mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush": mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush,
"fourone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush": fourone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush,
"fourone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush": fourone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush,
"fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card": fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card,
"fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card": fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card,
"fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush": fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush,
"fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush": fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush,
"threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush": threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush,
"threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush": threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush,
"threeoneone_flopturnriver_effective_hand_strengths_for_pairs": threeoneone_flopturnriver_effective_hand_strengths_for_pairs,
"twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards": twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards,
"twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards": twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards,
"twooneoneone_flopturnriver_effective_hand_strengths_for_pairs": twooneoneone_flopturnriver_effective_hand_strengths_for_pairs
}

In [5]:
import json

In [ ]:
for k,v in temp_dict.items():
    with open(f"poker/ehs/postriver/{k}.json", 'w') as f:
        json.dump(v, f)

In [ ]:
temp = mono_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush
mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush = {}
for k,v in temp.items():
    key = k.replace("s_", "o_")
    mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush[key] = v
with open(f"poker/ehs/postriver/mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush.json", 'w') as f:
    json.dump(mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush, f)

In [ ]:
temp = fourone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush
fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush = {}
for k,v in temp.items():
    key = k.replace("s_", "o_")
    fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush[key] = v
with open(f"poker/ehs/postriver/fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush.json", 'w') as f:
    json.dump(fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush, f)

In [ ]:
temp = threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush
threeoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush = {}
for k,v in temp.items():
    key = k.replace("s_", "o_")
    threeoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush[key] = v
with open(f"poker/ehs/postriver/threeoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush.json", 'w') as f:
    json.dump(threeoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush, f)

In [8]:
temp_list = [
"mono_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush",
"mono_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush",
"mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card",
"mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card",
"mono_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush",
"mono_flopturnriver_effective_hand_strengths_for_pairs_with_flush",
"mono_flopturnriver_effective_hand_strengths_for_pairs_no_flush",
"fourone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush",
"fourone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush",
"fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_higher_card",
"fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_with_flush_with_lower_card",
"fourone_flopturnriver_effective_hand_strengths_for_offsuit_cards_no_flush",
"fourone_flopturnriver_effective_hand_strengths_for_pairs_with_flush",
"fourone_flopturnriver_effective_hand_strengths_for_pairs_no_flush",
"threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_with_flush",
"threeoneone_flopturnriver_effective_hand_strengths_for_suited_cards_no_flush",
"threeoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards",
"threeoneone_flopturnriver_effective_hand_strengths_for_pairs",
"twooneoneone_flopturnriver_effective_hand_strengths_for_suited_cards",
"twooneoneone_flopturnriver_effective_hand_strengths_for_offsuit_cards",
"twooneoneone_flopturnriver_effective_hand_strengths_for_pairs",    
]

In [ ]:
ehs_postriver =  {}

In [11]:
for name in temp_list:
    with open(f"poker/ehs/postriver/{name}.json", "r") as f:
        temp = json.load(f)
        ehs_postriver = {
            **ehs_postriver,
            **temp
        }

In [15]:
with open("poker/ehs/postriver/ehs_postriver.json", "w") as f:
    json.dump(ehs_postriver, f)